In [1]:
import pandas as pd
import numpy as np
import os

import networkx as nx
import re


In [35]:
# importing data

edges = pd.read_csv('data/edges.csv')
nodes = pd.read_csv('data/nodes.csv')
hero_edges = pd.read_csv('data/hero-network.csv')
datasets = [edges, nodes, hero_edges]

In [25]:
problem2 = edges[ edges.hero.str.contains('ROGUE /')]
problem2

,hero,comic
67903,ROGUE /,A 10
67904,ROGUE /,A3 10
67905,ROGUE /,A 401
67906,ROGUE /,AF2 9
67907,ROGUE /,AF 33
...,...,...
68225,ROGUE /,XV.A 3
68226,ROGUE /,XV.A 4
68227,ROGUE /,XX 1
68228,ROGUE /,XX 2


In [57]:
problem = hero_edges[ hero_edges.hero2.str.contains('ROGUE')].hero2

In [58]:
problem

140       ROGUE /
148       ROGUE /
155       ROGUE /
161       ROGUE /
280       ROGUE /
           ...   
574451    ROGUE /
574456    ROGUE /
574460    ROGUE /
574463    ROGUE /
574465    ROGUE /
Name: hero2, Length: 3193, dtype: object

In [67]:
# preprocessing
def strip_rightend(string):
    sstring = '/'
    if string.endswith(sstring):
        string = re.sub(sstring, '', string)
    string = string.rstrip()


    if string == 'SPIDER-MAN/PETER PARKER':
        string = 'SPIDER-MAN/PETER PAR'

    return string

In [69]:
for ds in datasets:
    ds.dropna(inplace=True)
    for col in ds.columns:
        ds[col] = ds[col].apply(lambda row: strip_rightend(row))

In [82]:
hero_edges_arr = np.array(hero_edges)
hero_edges_arr.shape

(574467, 2)

In [83]:
idx_self = np.where(hero_edges_arr[:,0] == hero_edges_arr[:,1])
idx_self
# we check on a sample
hero_edges.iloc[8889]

hero1    MISS AMERICA/MADELIN
hero2    MISS AMERICA/MADELIN
Name: 8889, dtype: object

In [84]:
hero_edges_arr = np.delete(hero_edges_arr, idx_self, axis = 0) # delete the self referring rows by their index
print(hero_edges_arr.shape) # 574467 down to 572235
heroes_df = pd.DataFrame(hero_edges_arr) # ! new df 
heroes_df.columns = ['hero1', 'hero2']

(572235, 2)


creating the hero network graph 

In [86]:
heroes = nx.from_pandas_edgelist(heroes_df, 'hero1', 'hero2', create_using=nx.MultiGraph)

In [161]:
# heroes.edges
# heroes_df.apply(lambda row: heroes.add_edge(row['hero1'], row['hero2']), axis=1)
# lena exiwst already ! 

In [87]:
# counting edges for each node

nodes_edgecount = {}

for node in heroes.nodes():
    nodes_edgecount[node] = len(heroes.edges(node))
    
max_nr_edges = max(nodes_edgecount.values())

In [89]:
node_tuple_weight = {}
for e in heroes.edges: # .edges() doesnt have the edge key and believe this is the third tuple entry necessary
    weight = 1-(nodes_edgecount[e[0]]+nodes_edgecount[e[1]])/(2*max_nr_edges)
    node_tuple_weight[(e[0], e[1], e[2])] = weight

In [91]:
nx.set_edge_attributes(heroes, values = node_tuple_weight, name = 'weight')

In [92]:
# a = heroes.edges.data() # just don't try to show it, i don't know if its a setup in the networkx package but it will try and print the whole data and my kernel tends to stop

creating the comics (& corresponding heroes) graph

In [158]:
node_data = list(zip(nodes.node, [{'type':t}for t in nodes.type]))
edge_data = list(zip(edges.hero, edges.comic))

In [159]:
comics = nx.MultiGraph()
# nodes.apply(lambda row: comics.add_node(row.index, {'type': row.values()} ))
comics.add_nodes_from(node_data)
comics.add_edges_from(edge_data)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [133]:
# making list of top_n sorted heroes
ls = edges.groupby(by = 'hero')['comic'].size().sort_values(ascending=False)
top_n_heroes = np.array(list(zip(ls.index, ls.values)) )

## Functionality 3 - Shortest ordered Route

Input: graph, seq of superheroes h = [h_2, ..., h_n-1]
Initial node h_1 and an end node h_n
N:  top N heroes that their data sh

Output:

The shortest walk of comics that you need to read to get from hero_1 to hero_n
Considerations: For this functionality, you need to implement an algorithm that returns the shortest walk that goes from node h_j to h_n, which visits in order the nodes in h. The choice of h_j and h_n can be made randomly (or if it improves the performance of the algorithm, you can also define it in any other way)

Important Notes:

This algorithm should be run only on the second graph.
The algorithm needs to handle the case that the graph is not connected. Thus, only some of the nodes in h are reachable from h_1. In such a scenario, it is enough to let the program give in the output the string "There is no such path".
Since we are dealing with walks, you can pass on the same node h_i more than once, but you have to preserve order. E.g., if you start from Spiderman to reach deadpool, and your path requires you to visit iron-man and colossus, you can go back to any comics any time you want, assuming that the order in which you visit the heroes is still the same.
